In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/Desenvolvimento da Metodologia/SAUDA╠üVEIS/137/Segmentadas/PAC_45_DN0-esq.png
/kaggle/input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/Desenvolvimento da Metodologia/SAUDA╠üVEIS/137/Segmentadas/PAC_45_DN4-esq.png
/kaggle/input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/Desenvolvimento da Metodologia/SAUDA╠üVEIS/137/Segmentadas/PAC_45_DN11-esq.png
/kaggle/input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/Desenvolvimento da Metodologia/SAUDA╠üVEIS/137/Segmentadas/PAC_45_DN1-esq.png
/kaggle/input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/Desenvolvimento da Metodologia/SAUDA╠üVEIS/137/Segmentadas/PAC_45_DN1-dir.png
/kag

In [2]:
import os
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torchvision.models as models
from deap import base, creator, tools, algorithms
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from PIL import Image

In [3]:
# Preprocessing the data
training_positive = "/kaggle/input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/Desenvolvimento da Metodologia/DOENTES"
training_negative = "/kaggle/input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/Desenvolvimento da Metodologia/SAUDA╠üVEIS"
testing_positive = "/kaggle/input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/12 Novos Casos de Testes/DOENTES"
testing_negative = "/kaggle/input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/12 Novos Casos de Testes/SAUDA╠üVEIS"

In [4]:
train_images = []
train_labels = []
test_images = []
test_labels = []

In [5]:
for folder in os.listdir(training_positive):
    path = os.path.join(training_positive, folder, 'Segmentadas')
    for filename in os.listdir(path):
        if filename.endswith('.png'):
            img = Image.open(os.path.join(path, filename))
            train_images.append(np.array(img))
            train_labels.append(1)

for folder in os.listdir(training_negative):
    path = os.path.join(training_negative, folder, 'Segmentadas')
    for filename in os.listdir(path):
        if filename.endswith('.png'):
            img = Image.open(os.path.join(path, filename))
            train_images.append(np.array(img))
            train_labels.append(0)

In [6]:
for folder in os.listdir(testing_positive):
    path = os.path.join(testing_positive, folder, 'Segmentadas')
    for filename in os.listdir(path):
        if filename.endswith('.png'):
            img = Image.open(os.path.join(path, filename))
            test_images.append(np.array(img))
            test_labels.append(1)

for folder in os.listdir(testing_negative):
    path = os.path.join(testing_negative, folder, 'Segmentadas')
    for filename in os.listdir(path):
        if filename.endswith('.png'):
            img = Image.open(os.path.join(path, filename))
            test_images.append(np.array(img))
            test_labels.append(0)

In [7]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [8]:
print(f"Number of train scans: {len(train_images)} \n train labels: {len(train_labels)} ")
print(f"Number of test scans: {len(test_images)} \n test labels: {len(test_labels)} ")

Number of train scans: 1282 
 train labels: 1282 
Number of test scans: 240 
 test labels: 240 


In [9]:
# Pytorch dataset class for images
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        image = Image.fromarray(image)
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label, dtype=torch.float)
        return image, label


In [10]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [11]:
train_dataset = CustomDataset(images=train_images, labels=train_labels, transform=transform)
test_dataset = CustomDataset(images=test_images, labels=test_labels, transform=transform)
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [12]:
for batch in train_dataloader:
    data, labels =batch
    print("Data shape:", data.shape)
    print("Labels shape:", labels.shape)

Data shape: torch.Size([32, 1, 224, 224])
Labels shape: torch.Size([32])
Data shape: torch.Size([32, 1, 224, 224])
Labels shape: torch.Size([32])
Data shape: torch.Size([32, 1, 224, 224])
Labels shape: torch.Size([32])
Data shape: torch.Size([32, 1, 224, 224])
Labels shape: torch.Size([32])
Data shape: torch.Size([32, 1, 224, 224])
Labels shape: torch.Size([32])
Data shape: torch.Size([32, 1, 224, 224])
Labels shape: torch.Size([32])
Data shape: torch.Size([32, 1, 224, 224])
Labels shape: torch.Size([32])
Data shape: torch.Size([32, 1, 224, 224])
Labels shape: torch.Size([32])
Data shape: torch.Size([32, 1, 224, 224])
Labels shape: torch.Size([32])
Data shape: torch.Size([32, 1, 224, 224])
Labels shape: torch.Size([32])
Data shape: torch.Size([32, 1, 224, 224])
Labels shape: torch.Size([32])
Data shape: torch.Size([32, 1, 224, 224])
Labels shape: torch.Size([32])
Data shape: torch.Size([32, 1, 224, 224])
Labels shape: torch.Size([32])
Data shape: torch.Size([32, 1, 224, 224])
Labels sh

In [13]:
import torch.nn as nn
import torchvision.models as models
from torchvision.models import ResNet50_Weights

# Load the pretrained model
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)

# Modify the first convolutional layer to accept 1 channel instead of 3
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# Continue with the feature extraction or training process


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 167MB/s] 


In [14]:
def extract_features(dataloader):
    features = []
    labels = []
    
    with torch.no_grad():
        for images, label in dataloader:
            image_features = model(images)  # Extract features
            image_features = image_features.view(image_features.size(0), -1)  # Flatten features
            features.append(image_features)
            labels.append(label)
    
    return torch.cat(features), torch.cat(labels)

In [15]:
train_features, train_labels = extract_features(train_dataloader)
test_features, test_labels = extract_features(test_dataloader)

In [16]:
import random
from deap import base, creator, tools, algorithms
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer

# Assuming train_features, test_features, train_labels, and test_labels are already loaded as NumPy arrays

# Standardize the features
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

# Avoid overwriting the classes if they already exist
if not hasattr(creator, "FitnessMax"):
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximize accuracy
if not hasattr(creator, "Individual"):
    creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Create individuals with binary representation for feature selection (1: selected, 0: not selected)
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=train_features.shape[1])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Evaluation function
def evaluate(individual):
    selected_features = [index for index, selected in enumerate(individual) if selected == 1]
    
    if len(selected_features) == 0:
        return 0,  # No features selected
    
    # Train GradientBoostingClassifier on selected features
    clf = GradientBoostingClassifier(n_estimators=50)
    clf.fit(train_features[:, selected_features], train_labels)
    
    # Evaluate on test set
    predictions = clf.predict(test_features[:, selected_features])
    accuracy = accuracy_score(test_labels, predictions)
    return accuracy,

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.4)  # Blend crossover
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # Lower mutation probability
toolbox.register("select", tools.selTournament, tournsize=5)

# Create initial population
population = toolbox.population(n=25)

# Run the genetic algorithm
algorithms.eaSimple(population, toolbox, cxpb=0.8, mutpb=0.3, ngen=8, stats=None, verbose=True)

# Get the best individual (selected features)
best_individual = tools.selBest(population, 1)[0]
print("Best Individual (Feature Selection):", best_individual)

# Train classifier with selected features
selected_features = [index for index, selected in enumerate(best_individual) if selected == 1]
clf = GradientBoostingClassifier(n_estimators=50)
clf.fit(train_features[:, selected_features], train_labels)

# Predictions and performance metrics
predictions = clf.predict(test_features[:, selected_features])

accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions)
recall = recall_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions)

# Cross-validation for multiple metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}

# Cross-validation for multiple metrics
cv_results = cross_validate(
    clf, 
    train_features[:, selected_features], 
    train_labels, 
    cv=5, 
    scoring=scoring
)

# Print the cross-validated results
print(f"Cross-validated accuracy: {cv_results['test_accuracy'].mean():.4f}")
print(f"Cross-validated precision: {cv_results['test_precision'].mean():.4f}")
print(f"Cross-validated recall: {cv_results['test_recall'].mean():.4f}")
print(f"Cross-validated F1-score: {cv_results['test_f1'].mean():.4f}")


gen	nevals
0  	25    
1  	22    
2  	17    
3  	17    
4  	18    
5  	19    
6  	22    
7  	24    
8  	20    
Best Individual (Feature Selection): [-0.5542722813816208, 0.32515891631827143, 0.3306142336940186, 2.2830625601049874, 1.0, 0.0, -0.4548192750154942, 0.0, 0.13643186389587106, 0.0, 1.0, 1.0, 0.19677770765155397, 0.0, 1.0, 1.0, 1.2299501267288642, 1.0, 0.0, 1.2569056540042896, 1.0, 0.5130478045360105, 1.0, 1.0, 1.0, 1.3528788145866, 0.4004575929006614, 1.0, 0.974193814649914, 1.0, -0.10809965945230313, 1.0, 0.2716487702973121, -0.015331751695766765, 0.5312310475630128, 1.843142969727376, 0.9447879960801414, 1.0, 1.0, 0.038716774284649695, 0.4298547629003802, 1.0, 0.0, 1.2017010704734745, -0.49807425678527834, 0.0, -0.03306025451139716, -0.059409783047593034, 0.5739938369080197, 0.0, 0.5168605201607185, 1.0, 0.989961019060111, 0.9999999999999998, 0.09416728584568514, 1.2073497528052404, -0.6216231435056339, 0.0, 0.7375532558555082, 0.9096577437586386, 0.0, 1.0, 0.020035866829206